In [24]:
import time
import numpy as np
import pinocchio as pin
from example_robot_data import load
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


In [25]:
robot = load('ur5')
viz = MeshcatVisualizer(robot)
viz.display(pin.neutral(robot.model))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7012/static/


In [8]:
q = pin.neutral(robot.model)
v = np.zeros(robot.model.nv)
vq = np.zeros(robot.model.nv)

In [7]:
M = pin.crba(robot.model, robot.data, q)
b = pin.nle(robot.model, robot.data, q, vq)

In [107]:
q = pin.neutral(robot.model)
q = np.array([0,-np.pi/2,0,0,0,0])
vq = np.zeros(robot.model.nv)
viz.display(q)

dt = 2e-3
N_steps = 5000

for it in range(N_steps):
    t = it*dt

    # Compute the force that apply
    tauq = np.zeros(robot.model.nv)

    # Use aba forward dynamics to calculate aq
    aq = pin.aba(robot.model, robot.data, q, vq, tauq)

    # Double integration to update vq and q
    vq += aq * dt
    q = pin.integrate(robot.model, q, vq * dt)

    # Visualization
    if it%20==0: 
        viz.display(q)
        time.sleep(20*dt)

In [108]:
q = pin.neutral(robot.model)
q = np.array([0,-np.pi/2,0,0,0,0])
vq = np.zeros(robot.model.nv)
viz.display(q)

dt = 2e-3
N_steps = 5000
Kf = 0.3

for it in range(N_steps):
    t = it*dt

    # Retrieve the dynamics quantity at time t
    M = pin.crba(robot.model, robot.data, q)
    b = pin.nle(robot.model, robot.data, q, vq)

    # Compute the force that apply
    tauq = np.zeros(robot.model.nv)

    # Use generalized PFD to calculate aq
    aq = np.linalg.pinv(M) @ (tauq - Kf * vq - b)
    # Double integration to update vs and q
    vq += aq * dt
    q = pin.integrate(robot.model, q, vq * dt)

    # Visualization
    if it%20==0: 
        viz.display(q)
        time.sleep(20*dt)